## Imports

In [ ]:
import argparse
import os
import sys
import cv2
import matplotlib.pyplot as plt
import keras

In [ ]:
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.preprocessing.kitti import KittiGenerator
from keras_retinanet.preprocessing.open_images import OpenImagesGenerator
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.visualization import draw_annotations, draw_boxes
from keras_retinanet.utils.anchors import anchors_for_shape, compute_gt_annotations, AnchorParameters
from keras_retinanet.utils.config import read_config_file, parse_anchor_parameters

## Define Utility Functions

In [ ]:
def create_generator(dataset_type, annotations, classes, image_min_side, image_max_side, config):
    """ Create the data generators.

    Args:
        args: parseargs arguments object.
    """
    # create random transform generator for augmenting training data
    transform_generator = random_transform_generator(
        min_rotation=-0.1,
        max_rotation=0.1,
        min_translation=(-0.1, -0.1),
        max_translation=(0.1, 0.1),
        min_shear=-0.1,
        max_shear=0.1,
        min_scaling=(0.9, 0.9),
        max_scaling=(1.1, 1.1),
        flip_x_chance=0.5,
        flip_y_chance=0.5,
    )

    if dataset_type == 'coco':
        # import here to prevent unnecessary dependency on cocoapi
        from ..preprocessing.coco import CocoGenerator

        generator = CocoGenerator(
            args.coco_path,
            args.coco_set,
            transform_generator=transform_generator,
            image_min_side=args.image_min_side,
            image_max_side=args.image_max_side,
            config=args.config
        )
    elif dataset_type == 'pascal':
        generator = PascalVocGenerator(
            args.pascal_path,
            args.pascal_set,
            transform_generator=transform_generator,
            image_min_side=args.image_min_side,
            image_max_side=args.image_max_side,
            config=args.config
        )
    elif dataset_type == 'csv':
        generator = CSVGenerator(
            annotations,
            classes,
            transform_generator=transform_generator,
            image_min_side=image_min_side,
            image_max_side=image_max_side,
            config=config
        )
    elif args.dataset_type == 'oid':
        generator = OpenImagesGenerator(
            args.main_dir,
            subset=args.subset,
            version=args.version,
            labels_filter=args.labels_filter,
            parent_label=args.parent_label,
            annotation_cache_dir=args.annotation_cache_dir,
            transform_generator=transform_generator,
            image_min_side=args.image_min_side,
            image_max_side=args.image_max_side,
            config=args.config
        )
    elif args.dataset_type == 'kitti':
        generator = KittiGenerator(
            args.kitti_path,
            subset=args.subset,
            transform_generator=transform_generator,
            image_min_side=args.image_min_side,
            image_max_side=args.image_max_side,
            config=args.config
        )
    else:
        raise ValueError('Invalid data type received: {}'.format(dataset_type))

    return generator

def run(generator, random_transform, resize, draw_anchors, to_draw_annotations, anchor_params):
    """ Main loop.

    Args
        generator: The generator to debug.
        args: parseargs args object.
    """
    # display images, one at a time
    #for i in range(generator.size()):
    for i in range(200):
        # load the data
        image = generator.load_image(i)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        annotations = generator.load_annotations(i)
        if len(annotations['labels']) > 0 :
            # apply random transformations
#             if random_transform:
#                 image, annotations = generator.random_transform_group_entry(image, annotations)
            
            # resize the image and annotations
            if resize:
                image, image_scale = generator.resize_image(image)
                annotations['bboxes'] *= image_scale

            anchors = anchors_for_shape(image.shape, anchor_params=anchor_params)
            positive_indices, _, max_indices = compute_gt_annotations(anchors, annotations['bboxes'])

            # draw anchors on the image
            if draw_anchors:
                draw_boxes(image, anchors[positive_indices], (0, 0, 255), thickness=1)

            # draw annotations on the image
            if to_draw_annotations:
                # draw annotations in red
                draw_annotations(image, annotations, color=(255, 0, 0), label_to_name=generator.label_to_name)

                # draw regressed anchors in green to override most red annotations
                # result is that annotations without anchors are red, with anchors are green
                draw_boxes(image, annotations['bboxes'][max_indices[positive_indices], :], (0, 255, 0))

        plt.figure(figsize=(10, 10))
        plt.axis('off')
        plt.imshow(image)
        plt.show()

## Setup Paramters

In [ ]:
os.getcwd()

In [ ]:
dataset_type = 'csv'
annotations = ''  # path to training data csv file
classes = ''  # path to class label csv file


image_min_side = 800
image_max_side = 1333
config = None
random_transform = True
resize = True
draw_anchors = True
to_draw_annotations = True

## Setup Anchors (Optional)

In [ ]:
import numpy as np

In [ ]:
#Customized Anchor Settings:
sizes   = [32, 64, 128, 256, 512]
strides = [8, 16, 32, 64, 128]
ratios  = np.array([0.5, 1, 2], keras.backend.floatx())
scales  = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

In [ ]:
config = [ratios,scales,sizes,strides] #uncomment to setup anchors

## Run Debug

In [ ]:
# make sure keras is the minimum required version
check_keras_version()

# create the generator
generator = create_generator(dataset_type, annotations, classes, image_min_side, image_max_side, config)

# optionally load anchor parameters
anchor_params = None
if config:
    ratios,scales,sizes,strides = config
    anchor_params = AnchorParameters(sizes, strides, ratios, scales)

run(generator, random_transform, resize, draw_anchors, to_draw_annotations, anchor_params=anchor_params)


## End